<a href="https://colab.research.google.com/github/Aline-ACS/embedding_gemini_alura/blob/main/embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata

GOOGLE_API_KEY=userdata.get('secret_key')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [4]:
title="A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text=("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
"\n"
"Artigo completo: \n"
"\n"
"Gemini API & Google AI Studio; Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001", content=sample_text, title=title, task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.053263213, -0.038601145, -0.018003087, 0.01845275, 0.06680222, -0.005793013, -0.030032096, -0.01671813, 0.06428426, 0.06276712, 0.011168314, 0.013183347, -0.045070667, -0.022927193, 0.009059685, -0.024261309, 0.020752255, -0.010295092, -0.03583806, -0.00198773, 0.009129532, 0.0047538234, -0.030142428, -0.06761907, -0.01810743, 0.020307964, 0.017317705, -0.03780629, -0.03365574, 0.02841384, -0.05067973, 0.051613722, -0.03369966, 0.009258857, -0.03634504, -0.042909574, -0.025576683, -0.045004435, -0.0027551616, -0.003514414, 0.0067680134, -0.08446855, -0.010846663, 0.026456425, -0.004002681, -0.018905329, 0.046044894, 0.04473286, 0.015313287, -0.059458807, 0.028743314, 0.024830429, 0.06508537, -0.043034267, -0.0036022905, -0.007394243, 0.025496818, -0.03271884, 0.023018688, 0.00038309617, -0.0015459711, 0.022081804, -0.018675469, 0.05349867, 0.020770242, -0.055993073, -0.04277662, -0.00019961427, 0.011638818, 0.048534837, 0.009713548, 0.013991469, 0.060723245, -0.0643456

In [5]:
DOCUMENT1={"Título":"Operação do sistema de controle climático", "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro."}
DOCUMENT2={"Título":"Touchscreen", "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação."}
DOCUMENT3={"Título":"Mudança de marchas", "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada."}

documents=[DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [9]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df


,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [10]:
model = "models/embedding-001"

In [12]:
def embed_fn(title, text):
  return genai.embed_content(model=model, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [13]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.009572009, -0.017104592, -0.060387377, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.011367341, -0.048442047, -0.023845239, -0.0..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[0.00018316552, -0.03903683, 0.0017067897, 0.0..."


In [14]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)

  return df.iloc[indice]["Conteudo"]

In [18]:
consulta = "Como faço para trocar marchas em um carro do google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.


In [32]:
prompt= f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro")
response = model_2.generate_content(prompt)
print(response.text)

Seu Googlecar é tipo um carro esporte, mas sem a parte de acelerar forte. Ele tem uma transmissão automática maneirinha que te dá marchas sem você nem precisar pensar nelas. É só escolher a marcha que você quer e pronto!
